In [1]:
import socket
from datetime import datetime
import subprocess

# 간단한 서버 구축

## 기본

In [ ]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 12345))
server_socket.listen(0) # listen = 대기모드
print('listening')
client_socket, addr = server_socket.accept()
print('accepting')
data = client_socket.recv(65535) #데이터 송수신
print("receive:", data.decode())
client_socket.send(data)
print('send data')
client_socket.close()
print('종료')

## simple HTTP server

In [ ]:
#simple http server
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 80))
server_socket.listen(0)
print('listening')
client_socket, addr = server_socket.accept()
print('accepting')
data = client_socket.recv(65535)
print("receive:", data.decode())

client_socket.send('HTTP/1.0.200 OK\r\n\r\n<font color=magenta>Hello</font>'.encode("utf-8"))
# \r\n\r\n 필수
client_socket.close()

In [ ]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 80))
server_socket.listen(0)
print('listening')

while True:    # 기본적으로 작성하여 만든 서버는 1회성이기 때문에 루프를 통해 서버 상태 유지
    client_socket, addr = server_socket.accept()
    print('accepting')
    data = client_socket.recv(65535)
    print("receive:", data.decode())

    header = 'HTTP/1.0.200 OK\r\n\r\n'
    html = "hello" + str(datetime.now())
    client_socket.send(header.encode("utf-8"))
    client_socket.send(html.encode("utf-8"))
    client_socket.close()

## file 요청 및 응답 (정적 file)

### file 요청 및 응답 기본

In [ ]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 80))
server_socket.listen(0)
print('listening')

if True:  # 테스트 시의 편의성을 위해 while -> if 로 교체하여 일회성 서버 구현
    client_socket, addr = server_socket.accept()
    print('accepting')
    data = client_socket.recv(65535)
    data = data.decode()

    headers = data.split("\r\n")  # 요청시 송수신 데이터의 가장 첫부분이 요청 파일
    filename = headers[0].split(" ")[1]  # 데이터 스플릿, 파일 명 추출
    
    if '.html' in filename or '.txt' in filename:
        file = open("." + filename, 'rt', encoding='utf-8')  # 절대경로 상대경로화
        html = file.read()
        header = 'HTTP/1.0.200 OK\r\n\r\n'
        client_socket.send(header.encode("utf-8"))
        client_socket.send(html.encode("utf-8"))
        client_socket.close()
        
    elif '.jpg' in filename:
        client_socket.send('HTTP/1.1 200 OK\r\n'.encode())
        client_socket.send('Content-Type: image/jpg\r\n'.encode())
        client_socket.send('Accept-Range: bytes\r\n\r\n'.encode())
        file = open('.'+filename, 'rb')
        client_socket.send(file.read())
        file.close()
        
    elif '.png' in filename:
        client_socket.send('HTTP/1.1 200 OK\r\n'.encode())
        client_socket.send('Content-Type: image/png\r\n'.encode())
        client_socket.send('Accept-Range: bytes\r\n\r\n'.encode())
        file = open('.'+filename, 'rb')
        client_socket.send(file.read())
        file.close()
    else:
        header = 'HTTP/1.0. 404 File Not Found OK\r\n\r\n'
        client_socket.send(header.encode("utf-8"))
    client_socket.close()

In [ ]:
filename = '/index.html'  # 절대경로이기 때문에 수정 필요
# r : read, rt : read text, rb : read binary w : write
file = open("." + filename, 'rt', encoding='utf-8')
print(file.read())

### file 요청 및 응답 확장

In [ ]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 80))
server_socket.listen(0)
print('listening')

if True:  # 테스트 시의 편의성을 위해 while -> if 로 교체하여 일회성 서버 구현
    client_socket, addr = server_socket.accept()
    print('accepting')
    data = client_socket.recv(65535)
    data = data.decode()

    headers = data.split("\r\n")  # 요청시 송수신 데이터의 가장 첫부분이 요청 파일
    filename = headers[0].split(" ")[1]  # 데이터 스플릿, 파일 명 추출
    text_type = ['.html', '.txt']
    image_type = ['png', 'jpg', 'jpeg', 'gif']
    
    for i in text_type:        
        if i in filename:
            file = open("." + filename, 'rt', encoding='utf-8')  # 절대경로 상대경로화
            html = file.read()
            header = 'HTTP/1.0.200 OK\r\n\r\n'
            client_socket.send(header.encode("utf-8"))
            client_socket.send(html.encode("utf-8"))
            client_socket.close()
            
    for i in image_type:
        if i in filename:
            client_socket.send('HTTP/1.1 200 OK\r\n'.encode())
            client_socket.send(f'Content-Type: image/{i}\r\n'.encode())
            client_socket.send('Accept-Range: bytes\r\n\r\n'.encode())
            file = open('.'+filename, 'rb')
            client_socket.send(file.read())
            file.close()
        
    for a in text_type:
        for b in image_type:
            if a not in filename or b not in filename:
                header = 'HTTP/1.0. 404 File Not Found OK\r\n\r\n'
                client_socket.send(header.encode("utf-8"))
    client_socket.close()

### file 요청 및 응답 확장 2

In [ ]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 80))
server_socket.listen(0)
print("listening")

while  True :
    client_socket, addr = server_socket.accept()
    print("accepting")
    data = client_socket.recv(65535)    
    data = data.decode()
    print(data)        
    
    try :    
        headers = data.split("\r\n")
        filename = headers[0].split(" ")[1]

        if '.html' in filename:
            file = open("."+ filename, 'rt', encoding='utf-8')
            html = file.read()    
            header = 'HTTP/1.0 200 OK\r\n\r\n'        
            client_socket.send(header.encode("utf-8"))
            client_socket.send(html.encode("utf-8"))
            
        elif '.jpg' in filename :         
            client_socket.send('HTTP/1.1 200 OK\r\n'.encode())
            client_socket.send("Content-Type: image/jpg\r\n".encode())
            client_socket.send("Accept-Ranges: bytes\r\n\r\n".encode())
            file = open("." + filename, "rb")            
            client_socket.send(file.read())  
            file.close
            
        elif '.png' in filename :         
            client_socket.send('HTTP/1.1 200 OK\r\n'.encode())
            client_socket.send("Content-Type: image/png\r\n".encode())
            client_socket.send("Accept-Ranges: bytes\r\n\r\n".encode())
            file = open("." + filename, "rb")            
            client_socket.send(file.read())  
            file.close()
            
        else :
            header = 'HTTP/1.0 404 File Not Found\r\n\r\n'        
            client_socket.send(header.encode("utf-8"))
            
    except Exception as e :
        print(e)         
    client_socket.close()

## file 요청 및 응답 (동적 file)

In [11]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 80))
server_socket.listen(0)
print("listening")

if True :
    client_socket, addr = server_socket.accept()
    print("accepting")
    data = client_socket.recv(65535)    
    data = data.decode()
    #print(data)        
    
    try :    
        headers = data.split("\r\n")
        filename = headers[0].split(" ")[1]
        
        if '.py' in filename:
            html = subprocess.check_output(['python.exe', '.'+filename])
            html = html.decode('cp949')
            header = 'HTTP/1.0 200 OK\r\n\r\n'        
            client_socket.send(header.encode("utf-8"))
            #client_socket.send("Content-Type: text/html\r\n\r\n".encode())
            client_socket.send(html.encode("utf-8"))
            
        elif '.html' in filename:
            file = open("."+ filename, 'rt', encoding='utf-8')
            html = file.read()    
            header = 'HTTP/1.0 200 OK\r\n\r\n'        
            client_socket.send(header.encode("utf-8"))
            client_socket.send(html.encode("utf-8"))
            
        elif '.jpg' in filename or '.ico' in filename:         
            client_socket.send('HTTP/1.1 200 OK\r\n'.encode())
            client_socket.send("Content-Type: image/\r\n".encode())
            client_socket.send("Accept-Ranges: bytes\r\n\r\n".encode())
            file = open("." + filename, "rb")            
            client_socket.send(file.read())  
            file.close()               
            
        else :
            header = 'HTTP/1.0 404 File Not Found\r\n\r\n'        
            client_socket.send(header.encode("utf-8"))
    except Exception as e :
        print(e)         
    client_socket.close()

listening
accepting


python 내에서 dos 실행

In [ ]:
output = subprocess.check_output(['python.exe', 'test.py'])
print(output.decode('cp949'))